<a href="https://colab.research.google.com/github/ugundhar/sap_agentic_ai/blob/main/sap_agentic_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install requests hana-ml hdbcli "generative-ai-hub-sdk[all]" cryptography pypdf

In [2]:
from hdbcli import dbapi
import hana_ml.dataframe as dataframe
from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import TokenTextSplitter
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores.hanavector import HanaDB

In [3]:
from google.colab import files
uploaded = files.upload()

Saving irpa-r998-genaixl-sk.json to irpa-r998-genaixl-sk.json


In [8]:
#This script is used to get the access token for the BTP LLM API
import json
import requests

with open("irpa-r998-genaixl-sk.json", "r") as key_file:
    svcKey = json.load(key_file)
authUrl = svcKey["url"]
clientid = svcKey["clientid"]
clientsecret = svcKey["clientsecret"]
apiUrl = svcKey["serviceurls"]["AI_API_URL"]

# request token
params = {"grant_type": "client_credentials" }
resp = requests.post(f"{authUrl}/oauth/token",
                    auth=(clientid, clientsecret),
                    params=params)

BtpLlmApiUrl = apiUrl
BtpLlmAccessToken = resp.json()["access_token"]


In [9]:
# Set the environment variables for the BTP LLM API
import os

# Define Keys HERE
env_vars = {
 "AICORE_AUTH_URL": authUrl,
 "AICORE_CLIENT_ID": clientid,
 "AICORE_CLIENT_SECRET": clientsecret,
 "AICORE_RESOURCE_GROUP": "default",
 "AICORE_BASE_URL": apiUrl
}

os.environ.update(env_vars)

Retrieval-Augmented Generation (RAG)
In this section, we will explore Retrieval-Augmented Generation (RAG), a powerful technique for combining the strengths of retrieval and generation. RAG uses a combination of information retrieval and language models to generate responses based on retrieved content from external knowledge sources.

Why is RAG important?
RAG provides a solution for answering complex queries by:

Retrieving relevant information from large collections of unstructured data.
Generating a coherent and context-aware response based on retrieved content.
Use cases include:

Question answering systems.
Document summarization.
Chatbots that need up-to-date or specific external knowledge.
Motivating example
Large language models have not likely been trained on specific company information, espcially not company internal information. Like in the example below

In [13]:
from gen_ai_hub.proxy.native.openai import chat

messages = [ {"role": "user", "content": "Can I be reimboursed if I need to cancel my booked business trip because of personal reasons at SAP?"} ]

kwargs = dict(model_name='gpt-4', messages=messages)
response = chat.completions.create(**kwargs)
import json
response_dict = json.loads(response.json())
message_value = response_dict['choices'][0]['message']['content']
message_value

<ipython-input-13-a810070b0a86>:8: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  response_dict = json.loads(response.json())


"SAP's cancellation policy will largely depend on the specific travel policy of your SAP office, the terms of the trip booking, and possibly your reason for cancelling. In some cases, you might be reimbursed if you cancel far enough in advance, while in others, you may not be.\n\nKnowing the details of the situation, it's strongly advised that you discuss your cancellation and possible reimbursement with your manager, HR, or the person in charge of travel arrangements in your office to get a definitive answer. \n\nAlso, keep in mind that many travel insurance policies often cover unforeseen personal reasons that may require trip cancellation or interruption. Please check your SAP benefits which include travel insurance."

In [14]:
from google.colab import files

uploaded = files.upload()

Saving GlobalTravelPolicy (English).pdf to GlobalTravelPolicy (English).pdf
Saving SAP Global Car Fleet Policy_internal_ENGLISH.pdf to SAP Global Car Fleet Policy_internal_ENGLISH.pdf


In [15]:
#Policy download links- https://one.int.sap/company/policies_and_guidelines/travel_and_mobility#procurement___travel_b9ff/sap_global_car_fleet_policy_

from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Multiple documents on purpose - data
    PyPDFLoader("GlobalTravelPolicy (English).pdf"),
    PyPDFLoader("SAP Global Car Fleet Policy_internal_ENGLISH.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

Why Split Documents and Maintain Overlaps?

n retrieval-augmented generation (RAG), splitting large documents into smaller chunks is essential for efficient retrieval and better generation. Here's why:
# New
1. Memory Constraints: Large documents can be too big to process all at once in memory. Splitting into smaller, manageable chunks ensures the system can handle them efficiently.
2. Improved Retrieval: Splitting allows the retrieval engine to match relevant sections more accurately, rather than considering an entire large document.
3. Maintaining Context with Overlap: When splitting, overlaps between chunks help preserve context. This is important because the boundary of one chunk might split a coherent idea. The overlap helps ensure that the retrieval engine doesn’t lose important information that might span two chunks.

# Chunking Strategies
1. Fixed Size Chunking
Splits text into fixed-sized chunks based on tokens or characters.
Positives: Simple and easy to implement.
Negatives: Often breaks sentences or paragraphs, leading to incomplete context.
2. Sliding window Approch
Moves a fixed-size window across the text, creating overlapping chunks.
Positives: Maintains context across chunks.
Negatives: Introduces redundancy, increasing computational overhead.

3.Recursive Splitting


*   Divides text at semantic or structural boundaries like sentences or paragraphs.
*   Positives: Preserves meaning by aligning chunks with natural breaks.
*   Negatives: Computationally intensive and requires advanced parsing techniques.

Selecting the right chunking strategy is vital for the accuracy in outputof LLM-based applications. Fixed-size chunking offers simplicity but may lose meaning. Sliding windows maintain continuity but add redundancy. Recursive splitting ensures semantic integrity but requires careful implementation.


# Exercise: Explore Chunking with Overlaps

Let's experiment with chunking and overlaps. Adjust the chunk_size and chunk_overlap values in the code below and observe how the number of chunks changes.




In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#modify the chunk_size and chunk_overlap parameters

text_splitter= RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150
)

splits=text_splitter.split_documents(docs)
print(splits)
print(len(splits))

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2023-10-13T07:56:38+02:00', 'author': 'Stoehr, Iris', 'moddate': '2023-10-13T07:56:38+02:00', 'source': 'GlobalTravelPolicy (English).pdf', 'total_pages': 10, 'page': 0, 'page_label': '1'}, page_content='Version 5.3. September 1 2023  \n  Document Classification: INTERNAL \nThe online version of this document is the officially released version. \n                                                                                                                             Any copies or print-outs are not controlled  \n \n \nSAP GLOBAL TRAVEL POLICY \n \n \n \n \n \n \n \n \n  \nVersion Number: 5.3. \nEffective Date: September 1 2023 \nDocument Classification: INTERNAL'), Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2023-10-13T07:56:38+02:00', 'author': 'Stoehr, Iris', 'm

# Semantic Search and Distance Functions
Semantic search helps in retrieving documents that are semantically similar to a query, not just those with exact keyword matches. Common distance functions used in semantic search are:

1. L2 (Euclidean) Distance:
Measures the straight-line distance between two points (vectors). It's commonly used in geometric contexts but may not capture similarity well in high-dimensional spaces. When to use: • Image search and spatial data: L2 distance is often used when the data represents physical space or continuous attributes, such as in image search or geographical location data. In these cases, the exact “distance” between two points in the vector space is meaningful.

2. Cosine Similarity:
Measures the cosine of the angle between two vectors. This is often used in text retrieval tasks because it focuses on the direction rather than the magnitude of the vectors. When to use: • Text-based retrieval: Cosine similarity works well in natural language processing tasks, where the direction of the vector (representing the semantics of the text) matters more than its magnitude. • Document clustering: When clustering text documents or searching for documents with similar topics, cosine similarity helps retrieve semantically related information even if the length of the documents varies.

# Other possible distance measures include:


*   Manhattan Distance: Based on the sum of absolute differences. It works well when comparing structured (tabular) data with features that are unrelated or only loosely related.

*   List item: Measures the intersection over union of sets of tokens. Jaccard similarity is ideal for tasks where you compare sets of elements, such as keyword sets, tags, or lists. It measures how similar two sets are based on shared elements.

# Exercise: Compare Distance Metrics

Using the vectors below, calculate the L2 distance and cosine similarity between two text embeddings. Experiment with different metrics and see how the values change.







In [39]:
from gen_ai_hub.proxy.native.openai import embeddings

response = embeddings.create(
    input="yugandhar chaudhary",
    model_name="text-embedding-3-small"

)
print(response.data)

[Embedding(embedding=[-0.005370229948312044, -0.007774475030601025, -0.018489256501197815, -0.0025198031216859818, -0.0029611699283123016, 0.011671342886984348, 0.014444731175899506, -0.002038312144577503, -0.05254030227661133, -0.025563964620232582, 0.04157514497637749, -0.05500553548336029, 0.0029499351512640715, -0.011684182099997997, 0.0383652038872242, 0.010329587385058403, 0.0053349207155406475, 0.014996840618550777, 0.011266889981925488, 0.059525132179260254, -0.010515764355659485, 0.016024021431803703, 0.02963416837155819, 0.016190938651561737, 0.016434893012046814, 0.01698700338602066, 0.011221950873732567, 0.006224073935300112, 0.044810764491558075, -0.03582293167710304, -0.03970054164528847, -0.03582293167710304, 0.02372787892818451, -0.011786900460720062, 0.005062075797468424, 0.018925808370113373, -0.0027252391446381807, 0.0021105357445776463, 0.02539704740047455, 0.03353745490312576, -0.02714325487613678, -0.043757904320955276, 0.012249131686985493, 0.04414309933781624, 0

In [40]:
len(response.data[0].embedding)

1536

In [42]:
embedding_model = OpenAIEmbeddings(proxy_model_name='text-embedding-3-small')

In [74]:
sentence1 = "The Global Travel Policy is based on SAP’s values, strategies, and principles"
sentence2 = "Business trips are journeys to locations other than your regular place of work and home that fulfill a business purpose"
sentence3 = "SAP's Cloud Strategy is sound and customer adoption shows they value our company's offerings"

In [75]:
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

proxy_client = get_proxy_client('gen-ai-hub')
chat_llm = ChatOpenAI(proxy_model_name='gpt-4', proxy_client=proxy_client, temperature=0.0)

In [76]:
embedding1=embedding_model.embed_query(sentence1)
embedding2=embedding_model.embed_query(sentence2)
embedding3=embedding_model.embed_query(sentence3)
print(embedding1)
print(embedding2)
print(embedding3)
print(len(embedding1))
print(len(embedding2))
print(len(embedding3))

[-0.016352646052837372, 0.007334559690207243, 0.06922657042741776, 0.034074559807777405, 0.00033197019365616143, -0.03465817868709564, -0.03234613686800003, 0.021739928051829338, 0.01624041050672531, 0.01904628612101078, 0.010600599460303783, -0.0038356324657797813, -0.06567994505167007, -0.03775586560368538, -0.024377450346946716, -0.022918395698070526, -0.032391034066677094, -0.023232653737068176, 0.011784679256379604, -0.02388361655175686, 0.027677161619067192, 0.045769449323415756, -0.069675512611866, 0.0665329322218895, -0.03194209188222885, -0.013109052553772926, 0.005628587212413549, 0.0360274463891983, -0.05854179710149765, -0.04556742683053017, 0.037082456052303314, -0.0065208557061851025, 0.03921492397785187, -0.01704850234091282, 0.020280871540308, 0.023344887420535088, -0.01268255989998579, 0.06455759704113007, 0.023120418190956116, 0.001118843094445765, -0.020033953711390495, 0.020258424803614616, 0.024871284142136574, 0.010247059166431427, -0.022884724661707878, 0.0297198

NumPy's dot product function is a mathematical operation that takes two arrays of the same length and multiplies corresponding elements, followed by adding those products. In the context of comparing embeddings, it's extremely useful because it can measure the cosine similarity between two vectors when they're normalized. When embeddings (high-dimensional vectors that represent text) are compared, the angle between them can signify the degree of similarity. Smaller angles indicate more similar embeddings and larger angles suggest less similarity. By using the dot product, we effectively measure the cosine of the angle between two embeddings, giving us a quantifiable measure of their similarity. This is key in many natural language processing tasks, such as searching for documents similar to a given query in a large corpus.


In [77]:
import numpy as np


In [78]:
np.dot(embedding1,embedding2)

0.35451412912298347

In [79]:
np.dot(embedding1,embedding3)

0.5132870167805761

In [80]:
np.dot(embedding2,embedding3)

0.12859355265605338

Microsoft's Azure API is currently limiting the number of simultaneous calls to 16, therefore we need to process the splits in batches.

# Step 3: Setting up the vector store
a Vector store allows us to effciently store and retrieve document embeddings.dense vector representation of documents)

The Below code retrieves the values of HANA user, password and host name form environment variables, you can directly pass the same as well insted of first adding to the environment variables


In [ ]:
# get hana cloud username form environment variables

HANA_USER_DB= 'DBADMIN'

HANA_PASSWORD_VDB=''

#get the HANA cloud host form environment variables- you need to configure this with you instance details
#hint- This is ou SQL end point. But you need to remove":443 from the sql endpoint is already hardcoded
HANA_HOST=''

Establishes the connection to HANADB


In [ ]:
# user connection settings from the environment

connection= dbapi.connect(
    address=HANA_HOST,
    port=443,
    user=HANA_USER_DB,
    password=HANA_PASSWORD_VDB,
    encrypt='true',
    autocommit='true'
)

In [ ]:
#connect with the context

conn= dataframe.ConnectionContext(
    address=HANA_HOST,
    port=443,
    user=HANA_USER_DB,
    password=HANA_PASSWORD_VDB,
    encrypt='true',
    autocommit='true'
)

# import necessary langchain modules along with assignment of the proxy client as gen-api-hub & chat_ll_model initiation as gpt-4 which can be changed based on model availability in GENAI Hub


In [ ]:
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client


proxy_client = get_proxy_client('gen-ai-hub')
chat_llm = ChatOpenAI(proxy_model_name='gpt-4', proxy_client=proxy_client, temperature=0.0)

In [ ]:
db=HanaDB(
    embedding=embedding_model,
    table_name="TRAVEL_AND_ENVIRONMENT_POLICY",
    #vector column length=1536


)

## delete already existing documents from the table

db.delete(filter={})

## add the loaded document chunks

db.add_document(splits)

In [ ]:
# take a look at the table
hdf = conn.sql(''' SELECT "VEC_TEXT", "VEC_META", TO_NVARCHAR("VEC_VECTOR") AS "VEC_VECTOR" FROM "TRAVEL_AND_ENVIRONMENT_POLICY" ''')
df = hdf.head(10).collect()
df

In [ ]:
retriever = db.as_retriever()

# tep 5: Building the RAG Pipeline
Now that we have stored our document embeddings in HANA Vector Engine, let's build a simple retrieval-augmented generation pipeline using LangChain.

The process:


1.   Input a query.
2. Retrieve relevant documents based on the query.
3. Generate a response based on the retrieved content.





In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(""" provide answer based on the context provided

<content>
{context}
</content>

Question: {input}

""")



create document_chain with chat_llm(define earlier and prompt retrieval chain using hanadb and document chain

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(chat_llm,prompt)
retrieval_chain=create_retrieval_chain(retriever,document_chain)

invoke the query


In [ ]:
response = retrieval_chain.invoke({"input": "Which are the certified environmental and energy management systems that SAP run?"})
print(response["answer"])

# Limitation of Rag
1.Not Like Sql Queries: Rag systems don't behave like traditional SQL Queries. sal is deterministic, meaning that each query gives a precise. exact result from structured data. Rag system on the other hand, search over unstrcutred text and return approximate matches. They are probabilistic and can return different results for the same query.

2. Hallucinations: Rag systems can generate information that isn't present in the source document. this is knows as hallucination for example a rag system might generate details that seem plausible but are not supported by the retrieved documents

# Example of Hallucination in RAG

below is an example where the RAG system generates that contains hallucinated information

In [ ]:
response = retrieval_chain.invoke({"input": "Can you tell something about ISO 9001 mentioned in SAP policy"})
print(response["answer"])
# Note: There is no mention of ISO 9001 in the policies but it does exist otherwise. So lets see if it may result in hall

In [ ]:
response = retrieval_chain.invoke({"input": "How does ISO 9001 help SAP Achieve net-zero along SAP's value chain in line with a 1.5°C future by 2030"})
print(response["answer"])
# Note: There is no mention of ISO 9001 in the policies but it does exist otherwise. So lets see if it may result in hall

In [ ]:
response = retrieval_chain.invoke({"input": "Can you tell something about ISO 12345 mentioned in SAP policy"})
print(response["answer"])
# Note: There is no mention of ISO 12345 in the policies and it does not exist as well. So lets see if it may result in

In [ ]:
response = retrieval_chain.invoke({"input": "my son has the minimum age for driving a car in US and i would like to book a mid sized car for my business trip. Assuming I filled the gas, Is there any additional cost for him to be able to drive"})

print(response["answer"])

## Example of a hallucination

#STEP-6 Best Practices for RAG

To Improve the performance of RAG System, follow these best practices



*   Improve document quality: Ensure that the documents you retrieve from are clean and relevant.
*   Tune chunk sizes: Experiment with chunk sizes to balance retrieval accuracy and efficiency.
*   Reduce hallucinations: Implement filtering mechanisms to verify the accuracy of generated responses.

